# Topic Modeling Tutorial: LDA vs NMF (20 Newsgroups)

This notebook walks you through a **clean, fair comparison** of two popular topic modeling methods:

- **Latent Dirichlet Allocation (LDA)** — a **probabilistic** model that learns topics as word distributions and documents as mixtures of topics.
- **Non‑negative Matrix Factorization (NMF)** — a **linear algebra** approach that factorizes a document–term matrix into non‑negative parts (“topics”) and their document weights.

### What you will learn
1. How to build a **shared vocabulary** to make comparisons fair.
2. Why LDA uses **raw counts** and NMF prefers **TF‑IDF**.
3. How to examine **top words** per topic.
4. How to compute **two model‑agnostic quality metrics**:
   - **UMass coherence** (higher/less negative ≈ better)
   - **Mean Jensen–Shannon distance (JSD)** between topics (higher = more distinct)
5. How to interpret results and common trade‑offs.


In [ ]:
import numpy as np
from itertools import combinations

from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.decomposition import LatentDirichletAllocation, NMF
from sklearn.exceptions import ConvergenceWarning
import warnings

# For this demo we accept "good enough" convergence if top words are stable (which they are as you'll see)
warnings.filterwarnings("ignore", category=ConvergenceWarning)

## Step 1 — Configure the experiment

We set a few knobs to keep the run fast and reproducible while still informative:

- `n_samples`: how many documents we’ll use from the dataset.
- `n_topics`: how many topics to learn (a modeling choice; we’ll use 10).
- `n_top_words`: how many representative words to display per topic.
- `max_features`: cap the vocabulary size to control noise and runtime.
- `random_state`: fixed seed for repeatable results.


In [ ]:
n_samples    = 2000      # subset for speed
n_topics     = 10        # number of topics to learn
n_top_words  = 20        # words to display per topic
max_features = 5000      # vocabulary cap
random_state = 0         # reproducibility

## Step 2 — Load the dataset

We’ll use the **20 Newsgroups** corpus (a classic text dataset).  
To focus on message bodies (not metadata), we remove **headers, footers, and quoted text**.

> Outcome of this step: a list of raw documents (`docs`).

In [ ]:
ds = fetch_20newsgroups(shuffle=True, random_state=1,
                        remove=('headers', 'footers', 'quotes'))
docs = ds.data[:n_samples]
len(docs)

2000

## Step 3 — Build a **shared vocabulary** (fair comparison)

To compare LDA and NMF fairly, both must see **the same words**. We therefore:

1. Fit **one** `CountVectorizer` with identical rules (stop words, `min_df`, `max_df`, token pattern, and bigrams).
2. Use the resulting **counts matrix** (`X_counts`) as input to **LDA**.
3. Transform those same counts to **TF‑IDF** (`tfidf`) as input to **NMF**.

**Why these choices?**
- LDA’s generative assumptions expect **counts**.
- NMF typically behaves better on **TF‑IDF** (downweights ubiquitous words).

**Tokenizer settings**
- `token_pattern` drops numbers and 1‑character tokens.
- `ngram_range=(1,2)` includes both unigrams and bigrams for more context.
- `min_df` / `max_df` remove very rare and very common terms to reduce noise.


In [ ]:
cv = CountVectorizer(
    max_df=0.5, min_df=10, stop_words='english',
    token_pattern=r'(?u)\b[a-zA-Z][a-zA-Z]+\b',  # drop numbers / 1-char tokens
    ngram_range=(1, 2), max_features=max_features
)
X_counts = cv.fit_transform(docs)
feature_names = cv.get_feature_names_out()

tfidf = TfidfTransformer(norm='l2', sublinear_tf=True).fit_transform(X_counts)

X_counts.shape, tfidf.shape, len(feature_names)

((2000, 2513), (2000, 2513), 2513)

## Step 4 — Fit LDA (counts) and NMF (TF‑IDF)

### LDA (Latent Dirichlet Allocation)
- **Input**: `X_counts` (document–term counts).
- **Intuition**: each topic is a probability distribution over words; each document mixes topics with certain proportions.
- **Key params**:
  - `n_components=n_topics`: number of topics to discover.
  - `learning_method='online'`: efficient variational optimization.
  - `max_iter=10`: good enough for demos; more iterations refine topics.

### NMF (Non‑negative Matrix Factorization)
- **Input**: `tfidf` (document–term TF‑IDF weights).
- **Intuition**: factorize the matrix into **W (docs×topics)** and **H (topics×terms)** with non‑negative entries; rows of **H** act like topics.
- **Key params**:
  - `init='nndsvd'`: structured initialization that speeds convergence.
  - `solver='cd'`, `beta_loss='frobenius'`: coordinate descent on squared error.
  - `alpha_W=alpha_H=0.5`, `l1_ratio=1.0`: stronger L1 sparsity for **sharper, more distinct** topics.
  - `tol=1e-1`: “good‑enough” stopping; avoids chasing tiny changes.

**Loss choice (`beta_loss`)**
We set `beta_loss='frobenius'`, which makes NMF minimize squared error $|X - WH|_F^2$ (Frobenius norm).

* Works well with **TF-IDF** (real-valued, nonnegative features).
* Compatible with the fast coordinate-descent solver.
* Tends to produce clean, sparse “parts” when combined with L1 regularization (`l1_ratio=1.0`, `alpha_W/alpha_H`).

In [ ]:
lda = LatentDirichletAllocation(
    n_components=n_topics, max_iter=10,
    learning_method='online', learning_offset=50.,
    random_state=random_state
).fit(X_counts)

nmf = NMF(
    n_components=n_topics, init='nndsvd', solver='cd',
    beta_loss='frobenius',
    alpha_W=0.5, alpha_H=0.5, l1_ratio=1.0,  # strong L1 ⇒ sparse, distinct topics
    max_iter=2000, tol=1e-1,
    random_state=random_state
).fit(tfidf)

## Step 5 — Helper functions

We’ll need three utilities:

1. **`print_top_words`** — show the top‑`N` words for each topic.
2. **`umass_coherence`** — a quick **coherence** estimator using same‑corpus co‑occurrence:  
   $$ C_{UMass}(T) = \frac{1}{|P|} \sum_{(w_i,w_j)\in P} \log \frac{D(w_i, w_j) + \epsilon}{D(w_j) + \epsilon} $$
   where $D(w_i, w_j)$ counts documents containing both words, and $P$ iterates over pairs from a topic’s top words.
3. **`mean_js_distance`** — average **Jensen–Shannon divergence** between every pair of topic word distributions (higher = more distinct).

## UMass coherence

**What it asks:** *Do a topic’s top words tend to appear in the same documents?*

For each topic, take its top words and look up how often each pair co-occurs across documents in your corpus. Aggregate those pairwise signals into one score.

**How to read it:** Values are often **negative** (because of logarithms); **closer to 0 is better**. Higher/less-negative means the top words for a topic are found together more frequently, which usually aligns with human interpretability.

**Strengths/limits:** Fast and corpus-specific (a good thing), but sensitive to preprocessing and very common/rare words. It measures *internal consistency*, not uniqueness from other topics.

## Mean Jensen–Shannon distance (JSD) between topics

**What it asks:** *How different are the topics from each other?*

Treat each topic as a probability distribution over terms. Compute the Jensen–Shannon divergence for every pair of topics and average.

**How to read it:** **Higher means more distinct** topics (less vocabulary overlap). With natural logs, JSD ranges from **0** (identical) to about **0.69** (maximally different).

**Strengths/limits:** Captures *separation* between topics. It does **not** tell you if a topic is meaningful, only that topics differ.


In [ ]:
def print_top_words(model, feats, topn=20, title="Topics"):
    print(f"\n{title}:")
    for k, comp in enumerate(model.components_):
        top = np.argsort(comp)[-topn:][::-1]
        print(f"Topic #{k}:\n" + " ".join(feats[top]))

def _topn_indices(components, topn=20):
    return [np.argsort(c)[-topn:][::-1] for c in components]

def umass_coherence(components, X_counts, topn=20, eps=1e-12):
    # Binarize in a safe dtype (avoid int8 overflow)
    Xb = (X_counts > 0).astype(np.int32)
    # Co-doc counts in int64 to be safe
    C = (Xb.T @ Xb).astype(np.int64).tocsr()
    df = np.asarray(Xb.sum(axis=0)).ravel().astype(np.int64)

    topics = _topn_indices(components, topn)
    vals = []
    for ids in topics:
        s = 0.0; pairs = 0
        for i in range(1, len(ids)):
            wi = ids[i]
            for j in range(i):
                wj = ids[j]
                co = C[wi, wj]      # scalar int64
                denom = df[wj]
                s += np.log((co + eps) / (denom + eps))
                pairs += 1
        vals.append(s / max(pairs, 1))
    return float(np.mean(vals))

def mean_js_distance(components, eps=1e-12):
    # Topic separation: mean pairwise Jensen–Shannon divergence between topics
    P = components / (components.sum(axis=1, keepdims=True) + eps)
    def js(p, q):
        m = 0.5*(p+q)
        def kl(a, b):
            a = a + eps; b = b + eps
            return float((a*np.log(a/b)).sum())
        return 0.5*kl(p, m) + 0.5*kl(q, m)
    pairs = [js(P[i], P[j]) for i, j in combinations(range(P.shape[0]), 2)]
    return float(np.mean(pairs)) if pairs else 0.0

## Step 6 — Inspect the learned topics

We print the **top words** per topic for each model:

- **LDA**: words with the highest probability $p(w\mid z)$ for topic $z$.
- **NMF**: words with the largest weights in the topic’s component (the “parts” with strongest contribution).


In [ ]:
print_top_words(lda, feature_names, n_top_words, "Topics in LDA (counts)")
print_top_words(nmf, feature_names, n_top_words, "Topics in NMF (TF-IDF, Frobenius)")


Topics in LDA (counts):
Topic #0:
window food ago ve printer display just new hp win monitor years ago like using problem postscript font true fonts times
Topic #1:
drive disk card drives hard scsi controller speed rom bios hard disk floppy feature board supports interface bus mb pin bit
Topic #2:
banks gordon edu surrender pitt gordon banks intellect soon cadre skepticism shameful pitt edu geb dsl pitt dsl chastity chastity intellect edu shameful geb cadre shameful surrender
Topic #3:
people just don think like know god time good way say does make really right did didn want things going
Topic #4:
use key new like space know thanks used chip need work just does problem don power encryption keys clipper good
Topic #5:
cache ex meg powers simms ram set tv price guide high want problem output happening rest cycle normal turn operation
Topic #6:
edu windows mail com file graphics send version ftp available program software files contact pub pc image use server time
Topic #7:
people israel

### Let's estimate the topics!

* **#0 — Windows/printing & fonts**
  *window, printer, hp, postscript, font, monitor, display*
* **#1 — PC hardware: disks/SCSI/controllers**
  *drive, disk, scsi, controller, bios, floppy, board, bus*
* **#2 — Usenet signature/meme cluster (“Gordon Banks / pitt / geb / cadre”)**
  Artifacty block from repeated sig lines & threads.
* **#3 — General debate / opinion language**
  *people, just, don, think, know, time, right, want*
* **#4 — Crypto & Clipper chip / encryption**
  *key, encryption, clipper, keys, chip, power, use*
* **#5 — Performance & memory tuning**
  *cache, ram, simms, cycle, operation, output, set*
* **#6 — Software distribution & FTP**
  *windows, file, ftp, version, program, server, pc, image*
* **#7 — Geopolitics / Middle East & Armenia**
  *israel, armenian, greek, turkish, jews, war, israeli*
* **#8 — Motorcycles / DoD in-jokes / gear**
  *helmet, dod, radio, paint, cable, tx, star, copy*
* **#9 — Sports (NHL—Flyers) with a dash of civic/news terms**
  *team, flyers, division, play, gm vs* (+ a few government-ish strays)

# NMF (TF-IDF, Frobenius) topic labels (estimated)

* **#0 — General conversational filler / opinions**
  *just, like, don, know, think, good, way, make*
* **#1 — Windows & help/FTP/email**
  *windows, mail, file, card, drive, ftp, dos, version*
* **#2 — Usenet signature/meme cluster (pitt/geb/Gordon Banks)**
  Same artifact as LDA #2.
* **#3 — Religion/Christianity Q&A**
  *god, jesus, bible, christian*, plus “thanks/advance” email tone.
* **#4 — Sports & hobby chatter (hockey/baseball/bike/car)**
  *game, team, season, play, win, car, bike*
* **#5 — Crypto policy / Clipper & keys**
  *chip, government, key, encryption, clipper, public, security*
* **#6 — General internet/edu/com, lists & announcements**
  *edu, com, university, ftp, list, internet, space*
* **#7 — Hardware buy/sell & upgrades (with some religious bleed)**
  *drive, sale, hard, price, shipping, floppy* (plus stray *jesus, bible*)
* **#8 — Sports + computing/crypto overlap**
  *game, team, card, baseball, players, chip, clipper, drives*
* **#9 — Helpdesk / FTP how-to & “I need…” posts**
  *don, think, file, drive, ftp, internet, mail, send*

## Step 7 — Evaluate with model‑agnostic metrics

We summarize each model with two numbers:

1. **UMass coherence**: how often the top words of a topic co‑occur in documents.  
   Higher (less negative) generally means **more interpretable** topics.
2. **Mean JSD separation**: how far apart the topic distributions are from each other.  
   Higher means **more distinct** topics (less overlap).


In [ ]:
lda_coh = umass_coherence(lda.components_, X_counts, topn=n_top_words)
nmf_coh = umass_coherence(nmf.components_, X_counts, topn=n_top_words)

lda_js  = mean_js_distance(lda.components_)
nmf_js  = mean_js_distance(nmf.components_)

print("\n=== Summary Metrics ===")
print(f"LDA  coherence (UMass): {lda_coh:.4f}   | separation (JSD): {lda_js:.4f}")
print(f"NMF  coherence (UMass): {nmf_coh:.4f}   | separation (JSD): {nmf_js:.4f}")


=== Summary Metrics ===
LDA  coherence (UMass): -4.3433   | separation (JSD): 0.4010
NMF  coherence (UMass): -4.4337   | separation (JSD): 0.4413


## Step 8 — Interpret the results

Typical patterns you may observe:

- **Coherence**: LDA often edges NMF because its probabilistic assumptions favor word co‑occurrence within topics.
- **Separation (JSD)**: With strong sparsity, NMF topics can be more distinct (higher JSD).
- **Trade‑offs**: LDA topics may look slightly more diffuse but coherent; NMF topics can be sharper and easier to label, especially with TF‑IDF.

### Practical tips
- If topics look redundant, try adjusting `n_topics`, `min_df`, or `max_df`.
- If NMF warns about convergence but top words stabilize, it’s fine for interpretation.
- Keep the **shared vocabulary** so your comparison remains fair.

### Let's see the specifics here

* **Coherence (UMass)**: LDA −4.3433 vs NMF −4.4337: basically a tie, with a tiny edge to LDA (less negative = better). This matches intuition: LDA often groups co-occurring words a tad more cleanly.
* **Separation (JSD)**: LDA 0.4010 vs NMF 0.4413: NMF topics are **more distinct** from one another. That strong L1 sparsity did its job.

### What L1 sparsity does here

* **Zeros-out many weights.** Most entries in (W) and (H) become exactly 0.
* **Sharper topics.** Each topic uses a **small, distinctive** set of words → less vocabulary overlap between topics.
* **Sparser docs.** Each document loads on **fewer topics**.
* **Higher separation.** Because topics share fewer words, the **mean JSD** between topics goes up (we observed NMF JSD 0.4413 > LDA 0.4010).

### Trade-offs

* **Slower/never “fully” converging.** Heavy L1 makes coordinate descent keep “nibbling,” hence I had a ton of ConvergenceWarnings. (The topics still looked stable, so results were fine.)
* **Slightly lower coherence sometimes.** Strong sparsity can drop a few supportive words, so UMass may be a bit worse than LDA (as we saw: very close, slight LDA edge).

Step 9 — Check topic stability

To demonstrate robustness, fit NMF twice with different random seeds and compare the **overlap of top‑20 words** per topic. High overlap means stable topics.


In [ ]:
nmf_a = NMF(
    n_components=n_topics, init='nndsvd', solver='cd',
    beta_loss='frobenius',
    alpha_W=0.5, alpha_H=0.5, l1_ratio=1.0,
    max_iter=800, tol=1e-2, random_state=0
).fit(tfidf)

nmf_b = NMF(
    n_components=n_topics, init='nndsvd', solver='cd',
    beta_loss='frobenius',
    alpha_W=0.5, alpha_H=0.5, l1_ratio=1.0,
    max_iter=800, tol=1e-2, random_state=1
).fit(tfidf)

def avg_top_overlap(A, B, topn=20):
    overlaps = []
    for i in range(A.components_.shape[0]):
        ta = set(np.argsort(A.components_[i])[-topn:])
        tb = set(np.argsort(B.components_[i])[-topn:])
        overlaps.append(len(ta & tb) / topn)
    return float(np.mean(overlaps))

print("NMF topic stability (avg top-20 overlap):", avg_top_overlap(nmf_a, nmf_b))

NMF topic stability (avg top-20 overlap): 0.9350000000000002


* **NMF stability**: **0.935** average top-20 overlap across restarts is excellent. The NMF topics are highly stable.